# Data Quality Checks

This notebook performs basic data quality checks on the NYC 311 dataset.

In [2]:
import pandas as pd
import os

# Load data
df = pd.read_csv("/workspaces/nyc-311-ops-analysis/data/raw/311.csv")
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")

Total rows: 110,045
Total columns: 12


## Duplicate Check

In [3]:
# Check for duplicates on unique_key
duplicate_count = df['unique_key'].duplicated().sum()
total_unique = df['unique_key'].nunique()
print(f"Total records: {len(df):,}")
print(f"Unique keys: {total_unique:,}")
print(f"Duplicate keys: {duplicate_count:,}")
print(f"Duplicate percentage: {(duplicate_count/len(df)*100):.2f}%")

Total records: 110,045
Unique keys: 110,045
Duplicate keys: 0
Duplicate percentage: 0.00%


## Missing Values Summary

In [4]:
# Missing values summary
missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df) * 100).round(2),
    'Non-Null Count': df.notna().sum()
})
missing_summary = missing_summary.sort_values('Missing %', ascending=False)
print(missing_summary.to_string(index=False))

        Column  Missing Count  Missing %  Non-Null Count
   closed_date          23832      21.66           86213
          city           3808       3.46          106237
    descriptor           1329       1.21          108716
      latitude           1281       1.16          108764
     longitude           1281       1.16          108764
  incident_zip            921       0.84          109124
    unique_key              0       0.00          110045
  created_date              0       0.00          110045
        agency              0       0.00          110045
       borough              0       0.00          110045
complaint_type              0       0.00          110045
        status              0       0.00          110045


## Date Ranges

In [5]:
# Convert date columns and check ranges
df['created_date'] = pd.to_datetime(df['created_date'], errors='coerce')
df['closed_date'] = pd.to_datetime(df['closed_date'], errors='coerce')

date_ranges = pd.DataFrame({
    'Date Column': ['created_date', 'closed_date'],
    'Min Date': [df['created_date'].min(), df['closed_date'].min()],
    'Max Date': [df['created_date'].max(), df['closed_date'].max()],
    'Non-Null Count': [df['created_date'].notna().sum(), df['closed_date'].notna().sum()]
})
print(date_ranges.to_string(index=False))

 Date Column            Min Date            Max Date  Non-Null Count
created_date 2025-12-18 10:37:58 2025-12-31 02:21:14          110045
 closed_date 2025-12-17 07:00:00 2025-12-31 03:35:00           86213


## Save Summary Reports

In [6]:
# Create reports directory if it doesn't exist
os.makedirs("reports", exist_ok=True)

# Save summaries to CSV
missing_summary.to_csv("reports/missing_values_summary.csv", index=False)
date_ranges.to_csv("reports/date_ranges_summary.csv", index=False)

# Create overall quality summary
quality_summary = pd.DataFrame({
    'Metric': ['Total Rows', 'Unique Keys', 'Duplicate Keys', 'Duplicate %'],
    'Value': [
        len(df),
        total_unique,
        duplicate_count,
        f"{(duplicate_count/len(df)*100):.2f}%"
    ]
})
quality_summary.to_csv("reports/data_quality_summary.csv", index=False)

print("Summary reports saved to reports/ directory:")
print("  - missing_values_summary.csv")
print("  - date_ranges_summary.csv")
print("  - data_quality_summary.csv")

Summary reports saved to reports/ directory:
  - missing_values_summary.csv
  - date_ranges_summary.csv
  - data_quality_summary.csv
